In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [ ]:
def train(model, dev, epoch, batch_size, opt, train_loader, train_accuracy, train_losses, train_counter):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(dev), target.to(dev)
        opt.zero_grad()
        pred = model(data)
        loss = F.cross_entropy(pred, target)
        loss.backward()
        opt.step()
        pred = pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        if batch_idx % 50 == 0:
            accuracy = correct / ((batch_idx+1) * batch_size)
            train_accuracy.append(accuracy)
            train_losses.append(loss.item())
            train_counter.append(batch_idx*batch_size + (epoch - 1)*len(train_loader.dataset))
            print(f"""train epoch: {epoch} [{batch_idx * batch_size}/{len(train_loader.dataset)} ({(100. * batch_idx / len(train_loader)):.0f}%)]\tloss: {loss.item():.10f}\taccuracy: {accuracy:.10f}""")
    
def test(model, dev, batch_size, test_loader, test_accuracy, test_losses):
    model.eval()
    loss, accuracy, correct = 0, 0, 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(dev), target.to(dev)
            pred = model(data)
            loss += F.cross_entropy(pred, target).item()
            pred = pred.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    loss /= (len(test_loader.dataset) / batch_size)
    accuracy = correct / len(test_loader.dataset)
    test_accuracy.append(accuracy)
    test_losses.append(loss)
    print(f"\ntest set: average loss: {loss:.4f}, accuracy: {accuracy} ({(100. * accuracy):.0f}%)\n")
    

In [ ]:
from torchvision import datasets
from torchvision.transforms import v2
import torch.optim as optim

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'training on {dev} device')

# hyperparameter
batch_size = 64
epochs = 10
learning_rate = 0.005
momentum = 0.9

model = LeNet().to(dev)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

transforms = v2.Compose([v2.Pad(2), v2.ToImage(), v2.ToDtype(torch.float32, scale=True), v2.Normalize(mean=[0.1307], std=[0.3081])])
train_dataset = datasets.MNIST('./data', train=True, transform=transforms)
test_dataset = datasets.MNIST('./data', train=False, transform=transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

train_losses = []
train_counter = []
train_accuracy = []
test_losses = []
test_accuracy = []
test_counter = [i*len(train_loader.dataset) for i in range(1, epochs+1)]

for epoch in range(1, epochs+1):
    train(model, dev, epoch, batch_size, optimizer, train_loader, train_accuracy, train_losses, train_counter)
    test(model, dev, batch_size, test_loader, test_accuracy, test_losses)



In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(figsize=(18, 6), ncols=2)
# loss
ax1.plot(train_counter, train_losses, color='blue')
ax1.scatter(test_counter, test_losses, color='red')
ax1.legend(['train loss', 'test loss'], loc='upper right')
ax1.set_title('losses')
ax1.set_xlabel('number of training examples seen')
ax1.set_ylabel('cross entropy loss')
# accuracy
ax2.plot(train_counter, train_accuracy, color='blue')
ax2.scatter(test_counter, test_accuracy, color='red')
ax2.legend(['train accuracy', 'test accuracy'], loc='lower right')
ax2.set_title('accuracy')
ax2.set_xlabel('number of training examples seen')
ax2.set_ylabel('accuracy')
plt.show()